## Installing Libraries Needed

In [1]:
!python -m pip install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!python -m pip install openpyxl



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Dataset Inventory Loader

In [3]:
import os
import re

base_path = r"G:\My Drive\Labor Force Survey"

# Month ordering
month_order = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# Patterns
month_pattern = re.compile(
    r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)",
    re.IGNORECASE
)
year_pattern = re.compile(r"(20\d{2})")

# Detect year folders from drive
year_folders = [
    f for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f)) and f.isdigit()
]

print("Detected year folders:", sorted(year_folders))

inventory = {}

for year in sorted(year_folders):
    year_path = os.path.join(base_path, year)

    # Accept both CSV and XLSX
    data_files = [
        f for f in os.listdir(year_path)
        if f.lower().endswith(".csv") or f.lower().endswith(".xlsx")
    ]

    inventory[year] = {}

    for file in data_files:
        upper = file.upper()

        # Detect type
        if upper.endswith(".XLSX"):
            filetype = "metadata"  # XLSX = metadata
        else:
            filetype = "survey"    # CSV = survey

        # Detect month
        month_match = month_pattern.search(upper)
        month = (
            month_match.group(1).capitalize()
            if month_match
            else "Unmatched"
        )

        # Detect year inside filename
        year_match = year_pattern.search(upper)
        file_year = year_match.group(1) if year_match else "UNKNOWN"

        # Store into inventory
        if month not in inventory[year]:
            inventory[year][month] = []

        inventory[year][month].append({
            "filename": file,
            "filetype": filetype,
            "file_year": file_year
        })

# Print clean summary
print("\n=== DATASET INVENTORY SUMMARY ===\n")

for yr in sorted(inventory.keys()):
    print(f"Year {yr}:")

    sorted_months = sorted(
        inventory[yr].keys(),
        key=lambda m: month_order.get(m, 99)
    )

    for month in sorted_months:
        print(f"  {month}:")
        for item in inventory[yr][month]:
            print(f"    {item['filename']} ({item['filetype']})")

    print()


Detected year folders: ['2018', '2019', '2022', '2023', '2024']

=== DATASET INVENTORY SUMMARY ===

Year 2018:
  January:
    JANUARY_2018_METADATA.xlsx (metadata)
    JANUARY_2018.CSV (survey)
  April:
    APRIL_2018.CSV (survey)
    APRIL_2018_METADATA.xlsx (metadata)
  July:
    JULY_2018.CSV (survey)
    JULY_2018_METADATA.xlsx (metadata)
  October:
    OCTOBER_2018.CSV (survey)
    OCTOBER_2018_METADATA.xlsx (metadata)
  Unmatched:
    clean_jan_2018_metadata.csv (survey)

Year 2019:
  January:
    JANUARY_2019.CSV (survey)
    JANUARY_2019_METADATA.xlsx (metadata)
  April:
    APRIL_2019.CSV (survey)
    APRIL_2019_METADATA.xlsx (metadata)
  July:
    JULY_2019.CSV (survey)
    JULY_2019_METADATA.xlsx (metadata)
  October:
    OCTOBER_2019.CSV (survey)
    OCTOBER_2019_METADATA.xlsx (metadata)

Year 2022:
  January:
    JANUARY_2022.csv (survey)
    JANUARY_2022_METADATA.xlsx (metadata)
  February:
    FEBRUARY_2022.csv (survey)
    FEBRUARY_2022_METADATA.xlsx (metadata)
  March:

## Load Dataset Function

In [4]:
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load a dataset from the inventory.

    year: str, e.g., "2018"
    month: str, e.g., "January"
    filetype: "survey" or "metadata"
    sheet_number: 0(sheet 1) or 1(sheet 2)
    """
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])
    
    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)
    
    if sheet_number is not None:
        return pd.read_excel(file_path, sheet_name=sheet_number)
    
    return pd.read_excel(file_path)

## Metadata Sheet 1 Function 

In [5]:
def extract_variables(df):
    """
    Extract variable names and descriptions from metadata DataFrame.
    Looks for values in columns 'Unnamed: 4' (variable) and 'Unnamed: 5' (description).
    Returns a clean DataFrame with ['Variable', 'Description'].
    """

    # Keep only rows where column 4 has a variable name
    filtered = df[df['Unnamed: 4'].notna()][['Unnamed: 4', 'Unnamed: 5']]

    # Rename columns
    filtered.columns = ['Variable', 'Description']

    return filtered


In [6]:
import pandas as pd

# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 0)

# Call your function
variables_df = extract_variables(January_metadata)

# View results
variables_df.head()


,Variable,Description
1,PUFREG,Region
2,PUFPRV,Province
3,PUFPRRCD,Province Recode
4,PUFHHNUM,Household Unique Sequential Number
6,PUFURB2K10,2010Urban-RuralFIES


## Metadata Sheet 2 Function

In [7]:
# Load metadata
January_metadata = load_dataset("2018", "January", "metadata", 1)

January_metadata.head()

,PUFREG_VS1,Region,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,National Capital Region,13,NaN,NaN
1,NaN,NaN,Cordillera Administrative Region,14,NaN,NaN
2,NaN,NaN,Region I - Ilocos Region,1,NaN,NaN
3,NaN,NaN,Region II - Cagayan Valley,2,NaN,NaN
4,NaN,NaN,Region III - Central Luzon,3,NaN,NaN


In [12]:
import re
import pandas as pd

def process_metadata_sheet_binary1(df):
    """
    Strict horizontal reshape for metadata Sheet 2 (binary=1).
    Treats every row as data, carries forward variable/label context.

    Output columns:
      - Variable   (e.g., 'PUFREG_VS1')
      - Label      (e.g., 'Region')
      - Category   (e.g., 'National Capital Region', '00 - 04')
      - Code       (first numeric after category; 0 if none or range)
      - min_val    (range minimum; 0 if not a range)
      - max_val    (range maximum; 0 if not a range)
      - final_val  (equals Code for category-code rows; 0 for ranges)
    """

    # Drop only fully empty rows
    df = df.dropna(how="all").reset_index(drop=True)

    # Regex for variable tokens
    var_pat = re.compile(r"[A-Z0-9_]+_VS\d+", re.IGNORECASE)

    def to_str(x):
        if pd.isna(x) or x is None:
            return ""
        return str(x).strip()

    def is_num(x):
        return to_str(x).isdigit()

    def to_int(x, default=0):
        try:
            return int(to_str(x))
        except ValueError:
            return default

    def extract_ints(text):
        return [int(m.group()) for m in re.finditer(r"\d+", text)] if text else []

    def looks_like_range(text):
        s = text.lower()
        return "-" in s or "and over" in s or "to" in s

    records = []
    current_variable = ""
    current_label = ""

    for _, row in df.iterrows():
        # Keep empty cells, no filtering
        cells = [to_str(v) for v in row.tolist()]

        if not cells:
            continue

        # Forward-fill variable and label
        if len(cells) > 0 and var_pat.match(cells[0]):
            current_variable = cells[0]
            current_label = cells[1] if len(cells) > 1 else ""

        # Category = column 2
        category = cells[2] if len(cells) > 2 else ""
        if not category:
            continue

        # Code = column 3 safely
        code = to_int(cells[3]) if len(cells) > 3 else 0

        min_val = max_val = final_val = 0
        if code != 0:
            final_val = code

        records.append({
            "Variable": current_variable,
            "Label": current_label,
            "Category": category,
            "Code": code,
            "min_val": min_val,
            "max_val": max_val,
            "final_val": final_val
        })

    return pd.DataFrame(records)


In [13]:
# ==== Running the metadata ====

# January 2018, Sheet 2 (binary=1)
sheet2_jan2018 = load_dataset(year="2018", month="January", filetype="metadata", sheet_number=1)
sheet2_jan2018_reshaped = process_metadata_sheet_binary1(sheet2_jan2018)
print("January 2018 (Sheet 2) preview:")
sheet2_jan2018_reshaped.head(40)

January 2018 (Sheet 2) preview:


,Variable,Label,Category,Code,min_val,max_val,final_val
0,,,National Capital Region,13,0,0,13
1,,,Cordillera Administrative Region,14,0,0,14
2,,,Region I - Ilocos Region,1,0,0,1
3,,,Region II - Cagayan Valley,2,0,0,2
4,,,Region III - Central Luzon,3,0,0,3
5,,,Region IVA - CALABARZON,4,0,0,4
6,,,Region IVB - MIMAROPA,17,0,0,17
7,,,Region V- Bicol,5,0,0,5
8,,,Region VI - Western Visayas,6,0,0,6
9,,,Region VII - Central Visayas,7,0,0,7
